In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# Import necessary Libraries
import numpy as np
import os
import scipy.stats as st
import math
from sklearn import preprocessing
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
from math import exp
from math import log
import tensorflow as tf

import keras
from keras import layers
# from keras.utils import to_categorical
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense 
from keras.layers import MaxPooling1D, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D

In [ ]:
def DataPreparation(data_input_file):
  data = np.load(data_input_file, allow_pickle=True)
  # for k in data.files:
  #   print(k)
  X = data['X']
  # print(X.shape)
  X = X[:, 0, :, :]
  # print(X.shape)
  Y = data['y']
  # print(Y.shape)
  folds = data['folds']
  classes_number = Y.shape[1]
  Y = np.argmax(Y, axis=1)
  return X,Y,folds

In [ ]:
def DataPreparation_DNN(data_input_file):
  data = np.load(data_input_file, allow_pickle=True)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  n_class = Y.shape[1]
  folds = data['folds']
 
  return X,Y,folds,n_class

In [ ]:
# State-of-the-art ML models
def DTC():
  from sklearn.tree import DecisionTreeClassifier
  model1 = DecisionTreeClassifier()
  return model1

def KNC():
  from sklearn.neighbors import KNeighborsClassifier
  model2 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 3)
  return model2

def LRC():
  from sklearn.linear_model import LogisticRegression
  model3 = LogisticRegression(multi_class='ovr')
  return model3

def RForest():
  from sklearn.ensemble import RandomForestClassifier
  model4 = RandomForestClassifier(max_depth=2, random_state=0)
  return model4

def AdaBoost():
  from sklearn.ensemble import AdaBoostClassifier
  model5 = AdaBoostClassifier(n_estimators=150, random_state=0)
  return model5

def RBFC():
  model6 = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0,
      shrinking=True, probability=True, tol=0.001, cache_size=200,
      class_weight=None, verbose=False, max_iter=- 1, decision_function_shape='ovr',
      break_ties=False, random_state=None)
  return model6


# Deep Learning Models
def build_Lyu_DNN(row,col,num_classes):
  tf.keras.initializers.GlorotNormal(234)
  input_layer = tf.keras.Input(shape=(row,col,1,))
  layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())(input_layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.1)(layer)
  layer = tf.keras.layers.LSTM(28,return_sequences=True)(layer)
  layer = tf.keras.layers.Dropout(0.1)(layer)
  layerconv1 = tf.keras.layers.Conv1D(filters=30,kernel_size=5,  strides=1) (layer)
  layerconv1 = tf.keras.layers.Activation('relu')(layerconv1)
  layerconv2 = tf.keras.layers.Conv1D(filters=40,kernel_size= 10,strides=1) (layer)
  layerconv2 = tf.keras.layers.Activation('relu')(layerconv2)
  layerconv3 = tf.keras.layers.Conv1D(filters=50,kernel_size= 15,strides=1) (layer)
  layerconv3 = tf.keras.layers.Activation('relu')(layerconv3)
  layerconv4 = tf.keras.layers.Conv1D(filters=60,kernel_size= 20,strides=1) (layer)
  layerconv4 = tf.keras.layers.Activation('relu')(layerconv4)
  max1= tf.reduce_max(layerconv1, 1)
  max2= tf.reduce_max(layerconv2, 1)
  max3= tf.reduce_max(layerconv3, 1)
  max4= tf.reduce_max(layerconv4, 1)
  concat_layer = tf.keras.layers.concatenate([max1,max2,max3,max4],1)
  layer= tf.keras.layers.Dense(num_classes, activation = 'softmax')   (concat_layer) 

  model = tf.keras.Model(inputs=input_layer, outputs=layer)
  return model



def build_model_JGH(row,col,num_classes): 
  layers = [
      tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()),
      tf.keras.layers.LSTM(28,return_sequences=True,  input_shape=(row, col)),  
      tf.keras.layers.LSTM(28,return_sequences=True),                           
      tf.keras.layers.LSTM(28,return_sequences=True),                           
      tf.keras.layers.Dropout(0.1),
      tf.keras.layers.Conv1D(filters=64,kernel_size=5,strides=2,activation='relu'),
      tf.keras.layers.MaxPool1D(pool_size=2,strides = 2),
      tf.keras.layers.Conv1D(filters=128,kernel_size=3,strides=2,activation='relu'),
      tf.keras.layers.GlobalAveragePooling1D(),        
      tf.keras.layers.BatchNormalization(),     
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
       ]
      
  model = tf.keras.Sequential(layers)
  return model


def identity_block(input, kernel_size, filters, stage, block):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create layers
  output = keras.layers.Conv1D(filters1, 1, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  output = keras.layers.add([output, input])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output


def conv_block(input, kernel_size, filters, stage, block, strides=2):
  # Variables
  filters1, filters2, filters3 = filters
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'
  # Create block layers
  output = keras.layers.Conv1D(filters1, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '2a')(input)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2a')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters2, kernel_size, padding='same', kernel_initializer='he_normal', name=conv_name_base + '2b')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2b')(output)
  output = keras.layers.Activation('relu')(output)
  output = keras.layers.Conv1D(filters3, 1, kernel_initializer='he_normal', name=conv_name_base + '2c')(output)
  output = keras.layers.BatchNormalization(name=bn_name_base + '2c')(output)
  shortcut = keras.layers.Conv1D(filters3, 1, strides=strides, kernel_initializer='he_normal', name=conv_name_base + '1')(input)
  shortcut = keras.layers.BatchNormalization(name=bn_name_base + '1')(shortcut)
  output = keras.layers.add([output, shortcut])
  output = keras.layers.Activation('relu')(output)
  # Return a block
  return output
  
def build_model_ResNet(row, col, num_classes):
  input_shape = (row,col)
  # Create an input layer 
  input = keras.layers.Input(shape=input_shape)
  # Create output layers
  output = keras.layers.ZeroPadding1D(padding=2, name='padding_conv1')(input)                   #############
  output = keras.layers.Conv1D(64, 7, strides=2, use_bias=False, name='conv1')(output)          #############
  output = keras.layers.BatchNormalization(name='bn_conv1')(output)
  output = keras.layers.Activation('relu', name='conv1_relu')(output)
  output = keras.layers.MaxPooling1D(3, strides=2, padding='same', name='pool1')(output)
  output = conv_block(output, 3, [64, 64, 256], stage=2, block='a', strides=1)
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='b')
  output = identity_block(output, 3, [64, 64, 256], stage=2, block='c')
  output = conv_block(output, 3, [128, 128, 512], stage=3, block='a')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='b')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='c')
  output = identity_block(output, 3, [128, 128, 512], stage=3, block='d')
  output = conv_block(output, 3, [256, 256, 1024], stage=4, block='a')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='b')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='c')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='d')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='e')
  output = identity_block(output, 3, [256, 256, 1024], stage=4, block='f')
  output = conv_block(output, 3, [512, 512, 2048], stage=5, block='a')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='b')
  output = identity_block(output, 3, [512, 512, 2048], stage=5, block='c')
  output = keras.layers.GlobalAveragePooling1D(name='pool5')(output)
  output = keras.layers.Dense(num_classes, activation='softmax', name='fc1000')(output)
  # Create a model from input layer and output layers
  model = keras.models.Model(inputs=input, outputs=output)
  # # Print model
  # print()
  # print(model.summary(), '\n')
  # Compile the model
  # Return a model
  return model 

In [ ]:
# 15 features that we are extracting
def A(sample):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.average(sample[:,col])
      feat.append(average)

  return feat

def SD(sample):
  feat = []
  for col in range(0, sample.shape[1]):
      std = np.std(sample[:, col])
      feat.append(std)

  return feat

def AAD(sample):
  feat = []
  for col in range(0, sample.shape[1]):
      data = sample[:, col]
      add = np.mean(np.absolute(data - np.mean(data)))
      feat.append(add)

  return feat

def ARA(sample):
  #Average Resultant Acceleration[1]:
  # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
  feat = []
  sum_square = 0
  sample = np.power(sample, 2)
  for col in range(0, sample.shape[1]):
      sum_square = sum_square + sample[:, col]

  sample = np.sqrt(sum_square)
  average = np.average(sample)
  feat.append(average)
  return feat

def TBP(sample):
  from scipy import signal
  feat = []
  sum_of_time = 0
  for col in range(0, sample.shape[1]):
      data = sample[:, col]
      peaks = signal.find_peaks_cwt(data, np.arange(1,4))
      feat.append(peaks.tolist())
  return feat

def mean_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
    average = np.mean(sample[:,col])
    feat.append(average)

  return feat

def max_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amax(sample[:,col])
      feat.append(average)

  return feat

def min_ts(sample, Te=1.0): 
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amin(sample[:,col]) # min
      feat.append(average)

  return feat 

def skew_ts(sample, Te=1.0):    
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.skew(sample[:,col]) # skew
      feat.append(average)

  return feat

def kurtosis_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.kurtosis(sample[:,col]) # kurtosis 
      feat.append(average)

  return feat
 
def iqr_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.iqr(sample[:,col]) # interquartile rate
      feat.append(average)

  return feat

def mad_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.median(np.sort(abs(sample[:,col] - np.median(sample[:,col]))))
      feat.append(average)

  return feat

def area_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col], dx=Te)
      feat.append(average)

  return feat                            
 

def sq_area_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col] ** 2, dx=Te)
      feat.append(average)

  return feat
def energy_measure(sample):
  """Calculates energy measures"""
  feat = []
  for col in range(0,sample.shape[1]):

    em_x = np.mean(np.square(sample[:,col] ))
    feat.append(em_x)
  return feat

def stat_area_features(x, Te=1.0):
  mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
  max_ts = np.amax(x, axis=1).reshape(-1,1) # max
  min_ts = np.amin(x, axis=1).reshape(-1,1) # min
  std_ts = np.std(x, axis=1).reshape(-1,1) # std
  skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
  kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
  iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
  mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                              axis=1), axis=1).reshape(-1,1) # median absolute deviation
  area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
  sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

  return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                          iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)
  

def frequency_domain_features(x, Te=1.0):
  feat=[]
  # As the DFT coefficients and their corresponding frequencies are symetrical arrays
  # with respect to the middle of the array we need to know if the number of readings 
  # in x is even or odd to then split the arrays...
  if x.shape[1]%2 == 0:
      N = int(x.shape[1]/2)
  else:
      N = int(x.shape[1]/2) - 1
  xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
  dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
  
  # statistical and area features
  # dft_features = stat_area_features(dft, Te=1.0)
  # weighted mean frequency
  dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
  # 5 first DFT coefficients 
  dft_first_coef = dft[:,:5]    
  # 5 first local maxima of DFT coefficients and their corresponding frequencies
  dft_max_coef = np.zeros((x.shape[0],5))
  dft_max_coef_f = np.zeros((x.shape[0],5))
  for row in range(x.shape[0]):
      # finds all local maximas indexes
      extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
      # makes a list of tuples (DFT_i, f_i) of all the local maxima
      # and keeps the 5 biggest...
      extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                            key=operator.itemgetter(0), reverse=True)[:5] 
      for i, ext in enumerate(extrema_row):
          dft_max_coef[row,i] = ext[0]
          dft_max_coef_f[row,i] = ext[1]  

  feat.append(np.concatenate((dft_weighted_mean_f,dft_first_coef, dft_max_coef,dft_max_coef_f), axis=1).tolist() )
  # print(len(feat))
  # print(feat)
  return feat

## Feature Extraction

def feature_extraction(X):
  # Extracts the features, as mentioned by Catal et al. 2015
  # Average - A,
  # Standard Deviation - SD,
  # Average Absolute Difference - AAD,
  # Average Resultant Acceleration - ARA(1),
  # Time Between Peaks - TBP
  X_tmp = []
  
  for sample in X:
      features = A(sample)
      features = np.hstack((features, A(sample)))
      features = np.hstack((features, SD(sample)))
      features = np.hstack((features, AAD(sample)))
      features = np.hstack((features, ARA(sample)))
      """
      features = np.hstack((features, mean_ts(sample)))
      features = np.hstack((features, max_ts(sample)))
      features = np.hstack((features, min_ts(sample)))
      features = np.hstack((features, kurtosis_ts(sample)))
      features = np.hstack((features, iqr_ts(sample)))
      features = np.hstack((features, skew_ts(sample)))
      features = np.hstack((features, area_ts(sample)))
      features = np.hstack((features, sq_area_ts(sample)))
      features = np.hstack((features, mad_ts(sample)))
      features = np.hstack((features, mean_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, max_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, min_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, kurtosis_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, iqr_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, skew_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, area_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, sq_area_ts(sample[:,1:]-sample[:,:-1],1)))
      features = np.hstack((features, mad_ts(sample[:,1:]-sample[:,:-1],1)))
      """
      X_tmp.append(features)
      
  X = np.array(X_tmp)
  
  return X
  

In [ ]:
batch_size = 128
epochs = 100
learning_rate = 0.001

In [ ]:
# #For JSun
# def hidden_layer_generate(cnn_model,X):

#     """
#     CNNの中間層の出力を取得するモデルの構築
#     :param cnn_model: CNNモデル
#     :return:
#     """

#     layer_name = 'flatten_1'
#     hidden_layer_model = tf.keras.models.Model(inputs=cnn_model.input, outputs=cnn_model.layers[6].output)

#     cnn_train_result = hidden_layer_model.predict(X)

#     return hidden_layer_model, cnn_train_result
# def elm_model_generate(hidden_layer_cnn_lstm, y,x,num_classes):

#     """
#     ELMモデルの構築
#     """

#     target_train_oh = tf.keras.utils.to_categorical(y, num_classes)
#     print(target_train_oh.shape)

#     elm_model = hpelm.elm.ELM(hidden_layer_cnn_lstm.shape[1], num_classes)
#     print(hidden_layer_cnn_lstm.shape)
#     elm_model.add_neurons(128, func='sigm')

#     elm_model.train(hidden_layer_cnn_lstm, y, 'c')

#     return elm_model

# def predict_ELM(cnn_part, elm_part, X):

  

#   cnn_result = cnn_part.predict(X)
#   elm_result = elm_part.predict(cnn_result)
#   return elm_result    

In [ ]:
def Train(X_Train, Y_Train, X_Test, Y_Test, clf):
  confidence_factor_list = []
  clf.fit(X_Train, Y_Train)
  f1 = clf.predict_proba(X_Test)
  for i in range(0, f1.shape[0]):
    total_fuzzy_sum = sum(f1[i])
    confidence_factor_activity = f1[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array, Y_Test


def Train_Lyu(X, y, X_train, X_test, y_train, y_test, n_class):
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  model=build_Lyu_DNN(img_rows, img_cols, n_class)
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  model.compile(loss="categorical_crossentropy", optimizer=optimizer,  metrics=['accuracy'])
  model.fit(X_train, y_train, batch_size=batch_size,epochs=200,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array

  
def Train_JGH(X, y, X_train, X_test, y_train, y_test, n_class): 
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  model=build_model_JGH(img_rows,img_cols,n_class)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=250,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array


def Train_ResNet(X, y, X_train, X_test, y_train, y_test, n_class): 
  _,img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  model=build_model_ResNet(img_rows,img_cols,n_class)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=150,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array

In [ ]:
def Train_Lyu_Opp(X, Y, X_train, X_test, y_train, y_test, n_class):
  n_class = Y.shape[1]
  _, img_rows, img_cols = X_train.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
    
  model=build_Lyu_DNN(img_rows, img_cols, n_class)
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  model.compile(loss="categorical_crossentropy", optimizer=optimizer,  metrics=['accuracy'])
  model.fit(X_train, y_train, batch_size=batch_size,epochs=200,verbose=2)
  y_pred = model.predict(X_test)

  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array


def Train_JGH_Opp(X, Y, X_train, X_test, y_train, y_test, n_class):
  n_class = Y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape

  model=build_model_JGH(img_rows,img_cols,n_class)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=250,verbose=2)
  y_pred = model.predict(X_test)
  
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array



def Train_ResNet_Opp(X, Y, X_train, X_test, y_train, y_test, n_class):
  # n_class = Y.shape[1]
  _, img_rows, img_cols = X.shape
  X=X.reshape(X.shape[0],img_rows,img_cols,1)
  _,img_rows, img_cols,_ = X.shape
  model=build_model_ResNet(img_rows,img_cols,n_class)
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='ADAM')
  model.fit(X_train, y_train, batch_size=192, epochs=150,verbose=2)
  y_pred = model.predict(X_test)
  confidence_factor_list = []
  for i in range(0, y_pred.shape[0]):
    total_fuzzy_sum = sum(y_pred[i])
    confidence_factor_activity = y_pred[i]/total_fuzzy_sum
    confidence_factor_list.append(confidence_factor_activity)
  confidence_factor_array = np.asarray(confidence_factor_list)
  return confidence_factor_array

In [ ]:
def PredictClass_5(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, merged_fuzzy_rank_array_3, merged_fuzzy_rank_array_4, merged_fuzzy_rank_array_5,
                confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3, confidence_factor_array_4, confidence_factor_array_5):  

  Y_Pred = []
  RSc = {}
  CFSc = {}
  FSc = {}
  k = 3#int((fuzzy_rank_array_1.shape[1])/2);
  PcR = 0.888
  PcCF = 0
    
  for i in range(0, len(fuzzy_rank_array_1)):
    item_1 = merged_fuzzy_rank_array_1[i]
    item_2 = merged_fuzzy_rank_array_2[i]
    item_3 = merged_fuzzy_rank_array_3[i]
    item_4 = merged_fuzzy_rank_array_4[i]
    item_5 = merged_fuzzy_rank_array_5[i]
    indices_1 = sorted(range(len(item_1)), key=lambda x: item_1[x])[0:k]
    indices_2 = sorted(range(len(item_2)), key=lambda x: item_2[x])[0:k]
    indices_3 = sorted(range(len(item_3)), key=lambda x: item_3[x])[0:k]
    indices_4 = sorted(range(len(item_4)), key=lambda x: item_4[x])[0:k]
    indices_5 = sorted(range(len(item_5)), key=lambda x: item_5[x])[0:k]

    for j in range(0, len(item_1)):
      if j in indices_1:
        RSc[j] = item_1[j]
        CFSc[j] = 1-((1/5)*confidence_factor_array_1[i][j])
      else:
        RSc[j] = PcR
        CFSc[j] = 1-((1/5)*PcCF)
      
      if j in indices_2:
        RSc[j] += item_2[j]
        CFSc[j] += -((1/5)*confidence_factor_array_2[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/3)*PcCF)

      if j in indices_3:
        RSc[j] += item_3[j]
        CFSc[j] += -((1/5)*confidence_factor_array_3[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/5)*PcCF)

      if j in indices_4:
        RSc[j] += item_4[j]
        CFSc[j] += -((1/5)*confidence_factor_array_4[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/5)*PcCF)

      if j in indices_5:
        RSc[j] += item_5[j]
        CFSc[j] += -((1/5)*confidence_factor_array_5[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/5)*PcCF)

    CFSc_keys = [*CFSc]
    # print("Keys:", CFSc_keys)
    # for key in CFSc_keys:
    #   CFSc[key] = (1 - CFSc[key])
    # print("RSc: ", RSc)
    # print("CFSc: ", CFSc)
    valid_keys = list(set([*RSc]) & set([*CFSc]))
    for key in valid_keys:
      FSc[key]= RSc[key]* CFSc[key]
    # print("FSc: ", FSc)
    predicted_class = min(FSc, key=FSc.get)
    Y_Pred.append(predicted_class)
  return np.asarray(Y_Pred)


In [ ]:
def PredictClass_3(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, merged_fuzzy_rank_array_3,
                confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3):  

  Y_Pred = []
  RSc = {}
  CFSc = {}
  FSc = {}
  k = 3#int((fuzzy_rank_array_1.shape[1])/2);
  PcR = 0.888
  PcCF = 0
    
  for i in range(0, len(fuzzy_rank_array_1)):
    item_1 = merged_fuzzy_rank_array_1[i]
    item_2 = merged_fuzzy_rank_array_2[i]
    item_3 = merged_fuzzy_rank_array_3[i]
    indices_1 = sorted(range(len(item_1)), key=lambda x: item_1[x])[0:k]
    indices_2 = sorted(range(len(item_2)), key=lambda x: item_2[x])[0:k]
    indices_3 = sorted(range(len(item_3)), key=lambda x: item_3[x])[0:k]

    for j in range(0, len(item_1)):
      if j in indices_1:
        RSc[j] = item_1[j]
        CFSc[j] = 1-((1/3)*confidence_factor_array_1[i][j])
      else:
        RSc[j] = PcR
        CFSc[j] = 1-((1/3)*PcCF)
      
      if j in indices_2:
        RSc[j] += item_2[j]
        CFSc[j] += -((1/3)*confidence_factor_array_2[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/3)*PcCF)

      if j in indices_3:
        RSc[j] += item_3[j]
        CFSc[j] += -((1/3)*confidence_factor_array_3[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/3)*PcCF)


    CFSc_keys = [*CFSc]
    # print("Keys:", CFSc_keys)
    # for key in CFSc_keys:
    #   CFSc[key] = (1 - CFSc[key])
    # print("RSc: ", RSc)
    # print("CFSc: ", CFSc)
    valid_keys = list(set([*RSc]) & set([*CFSc]))
    for key in valid_keys:
      FSc[key]= RSc[key]* CFSc[key]
    # print("FSc: ", FSc)
    predicted_class = min(FSc, key=FSc.get)
    Y_Pred.append(predicted_class)
  return np.asarray(Y_Pred)

In [ ]:
def PredictClass_2(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, confidence_factor_array_1, confidence_factor_array_2):  

  Y_Pred = []
  RSc = {}
  CFSc = {}
  FSc = {}
  k = 3#int((fuzzy_rank_array_1.shape[1])/2);
  PcR = 0.888
  PcCF = 0
    
  for i in range(0, len(fuzzy_rank_array_1)):
    item_1 = merged_fuzzy_rank_array_1[i]
    item_2 = merged_fuzzy_rank_array_2[i]
    indices_1 = sorted(range(len(item_1)), key=lambda x: item_1[x])[0:k]
    indices_2 = sorted(range(len(item_2)), key=lambda x: item_2[x])[0:k]

    for j in range(0, len(item_1)):
      if j in indices_1:
        RSc[j] = item_1[j]
        CFSc[j] = 1-((1/2)*confidence_factor_array_1[i][j])
      else:
        RSc[j] = PcR
        CFSc[j] = 1-((1/2)*PcCF)
      
      if j in indices_2:
        RSc[j] += item_2[j]
        CFSc[j] += -((1/2)*confidence_factor_array_2[i][j])
      else:
        RSc[j] += PcR
        CFSc[j] += -((1/2)*PcCF)

    CFSc_keys = [*CFSc]
    # print("Keys:", CFSc_keys)
    # for key in CFSc_keys:
    #   CFSc[key] = (1 - CFSc[key])
    # print("RSc: ", RSc)
    # print("CFSc: ", CFSc)
    valid_keys = list(set([*RSc]) & set([*CFSc]))
    for key in valid_keys:
      FSc[key]= RSc[key]* CFSc[key]
    # print("FSc: ", FSc)
    predicted_class = min(FSc, key=FSc.get)
    Y_Pred.append(predicted_class)
  return np.asarray(Y_Pred)


In [ ]:
# Functions
from math import exp
from math import log
    
def mitcherlich(time, alpha, beta, rate):
    """
    time : time
    alpha : upper asymptote
    beta : growth range
    rate : growth rate
    """
    result = alpha - beta * rate ** time
    return result

def blumberg(time, alpha, slope, w0=1):
    """
    time : time
    alpha : upper asymptote
    w0 : a reference value at time = time_0
    slope : slope of growth
    """
    result = (alpha * (time) ** slope) / (w0 + (time) ** slope)
    return result

def gompertz(time, alpha, beta, rate):
    """
    time : time
    alpha : upper asymptote
    beta : growth displacement
    rate : growth rate
    """
    result = alpha * exp(-beta * exp(-rate * time))
    return result

def weibull(time, alpha, beta, rate, slope):
    """
    time : time
    alpha : upper asymptote
    beta : growth displacement
    rate : growth rate
    slope : slope of growth
    """
    result = alpha - beta * exp(-rate * time ** slope)
    return result


In [ ]:
def GenerateFuzzyRank_Gaussian(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = 1-math.exp(((confidence_factor-1.0)*(confidence_factor-1.0)/(-2.0)))
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)
  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Mitcherlich(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = mitcherlich(confidence_factor,2,1,2)
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)
  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Blumberg(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = 1 - blumberg(confidence_factor, 1, 0.0001, w0=0.5)
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)
  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Gompertz(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = 1 - gompertz(confidence_factor, 1, 2, 3)
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)
  return np.asarray(fuzzy_rank_list)

def GenerateFuzzyRank_Weibull(confidence_factor_array):  
  fuzzy_rank_list = []
  for item in confidence_factor_array:
    arr=[]
    for confidence_factor in item:
      fuzzy_rank = -weibull(confidence_factor, 0, 0.5, 2, 2) # We negate this 
      arr.append(fuzzy_rank)
    fuzzy_rank_list.append(arr)
  return np.asarray(fuzzy_rank_list)


In [ ]:
def MergeFuzzyRanks(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3, fuzzy_rank_array_4, fuzzy_rank_array_5, confidence_factor_array):
  k = 3
  merged_fuzzy_rank_array = []
  for i in range(len(fuzzy_rank_array_1)):
    combined_fuzzy_scores = []
    topKindices = set(sorted(range(len(confidence_factor_array[i])), key=lambda x: confidence_factor_array[i][x])[len(confidence_factor_array)-k-1:-1])
    # set_topKindices = set(topKindices)
    for j in range(len(fuzzy_rank_array_1[i])):  
      if j in topKindices:
        fuzzy_score = min(fuzzy_rank_array_1[i][j], fuzzy_rank_array_2[i][j], fuzzy_rank_array_3[i][j], fuzzy_rank_array_4[i][j], fuzzy_rank_array_5[i][j])
        # print("Fuzzy Score: ", fuzzy_score)
      else:
        fuzzy_score = max(fuzzy_rank_array_1[i][j], fuzzy_rank_array_2[i][j], fuzzy_rank_array_3[i][j], fuzzy_rank_array_4[i][j], fuzzy_rank_array_5[i][j])
        # print("Fuzzy Score: ", fuzzy_score)
      combined_fuzzy_scores.append(fuzzy_score)
    # print("Combined Fuzzy: ", combined_fuzzy_scores)
    merged_fuzzy_rank_array.append(combined_fuzzy_scores)
  return np.asarray(merged_fuzzy_rank_array)



In [ ]:
# # For ensembles (except Opportunity): -- Uncomment and Test!!

# if __name__ == "__main__":  
#   X,Y,folds = DataPreparation('/content/drive/MyDrive/data/SNOW/WISDM.npz')
#   clf1 = DTC()
#   clf2 = KNC()
#   clf3 = LRC()
#   clf4 = RForest()
#   clf5 = AdaBoost()
#   clf6 = RBFC()

#   avg_acc_list = []
#   avg_recall_list = []
#   avg_f1_list = []

#   for i in range(len(folds)):
#     Train_idx = folds[i][0]
#     Test_idx = folds[i][1]
#     # X_Train_nonnormalized = X[Train_idx]
#     X_Train = X[Train_idx]
#     Y_Train = Y[Train_idx]
#     # X_Test_nonnormalized = X[Test_idx]
#     X_Test = X[Test_idx]
#     Y_Test = Y[Test_idx]

#     X_Train = feature_extraction(X_Train)
#     # X_Train = preprocessing.normalize(X_Train_nonnormalized, norm='l2')
#     X_Test = feature_extraction(X_Test)
#     # X_Test = preprocessing.normalize(X_Test_nonnormalized, norm='l2')

#     ## Train_idx = folds[i][0]
#     ## Test_idx = folds[i][1]
#     ## X_Train = X[Train_idx]
#     ## Y_Train = Y[Train_idx]
#     ## X_Test = X[Test_idx]
#     ## Y_Test = Y[Test_idx]
#     ## print(X_Test.shape[0])
#     ## X_Train = feature_extraction(X_Train)
#     ## X_Test = feature_extraction(X_Test)

#     # For classifier 1
#     confidence_factor_array_1, Y_Test = Train(X_Train, Y_Train, X_Test, Y_Test, clf1)
#     fuzzy_rank_array_1 = GenerateFuzzyRank_Gaussian(confidence_factor_array_1)
#     fuzzy_rank_array_2 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_1)
#     fuzzy_rank_array_3 = GenerateFuzzyRank_Blumberg(confidence_factor_array_1)
#     fuzzy_rank_array_4 = GenerateFuzzyRank_Gompertz(confidence_factor_array_1)
#     fuzzy_rank_array_5 = GenerateFuzzyRank_Weibull(confidence_factor_array_1)

#     # print(fuzzy_rank_array_1[0])
#     # print(fuzzy_rank_array_2[0])
#     # print(fuzzy_rank_array_3[0])
#     # print(fuzzy_rank_array_4[0])
#     # print(fuzzy_rank_array_5[0])
#     # print(confidence_factor_array_1)
#     # break

#     ######
#     # print("\nCLASSIFIER 1")
#     # for row in fuzzy_rank_array_1:
#     #   print("Gaussian: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_2:
#     #   print("Mitcherlich: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_3:
#     #   print("Blumberg: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_4:
#     #   print("Gompertz: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_5:
#     #   print("Weibull: ",row)
#     #   break
#     ######
     
    
#     # print("Gaussian: ", fuzzy_rank_array_1)
#     # print("Mitcherlich: ", fuzzy_rank_array_2)
#     # print("Blumberg: ",fuzzy_rank_array_3)
#     # print("Gompertz: ", fuzzy_rank_array_4)
#     # print("Richard: ", fuzzy_rank_array_5)
#     # print("\n")


#     merged_fuzzy_rank_array_1 = MergeFuzzyRanks(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3, 
#                                               fuzzy_rank_array_4, fuzzy_rank_array_5, confidence_factor_array_1)
#     # print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_1)

  
#     # For classifier 2
#     confidence_factor_array_2, Y_Test = Train(X_Train, Y_Train, X_Test, Y_Test, clf2)
#     fuzzy_rank_array_6 = GenerateFuzzyRank_Gaussian(confidence_factor_array_2)
#     fuzzy_rank_array_7 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_2)
#     fuzzy_rank_array_8 = GenerateFuzzyRank_Blumberg(confidence_factor_array_2)
#     fuzzy_rank_array_9 = GenerateFuzzyRank_Gompertz(confidence_factor_array_2)
#     fuzzy_rank_array_10 = GenerateFuzzyRank_Weibull(confidence_factor_array_2)
    
#     ######
#     # print("\nCLASSIFIER 2")
#     # for row in fuzzy_rank_array_6:
#     #   print("Gaussian: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_7:
#     #   print("Mitcherlich: ",row)
#     #   break
     
#     # for row in fuzzy_rank_array_8:
#     #   print("Blumberg: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_9:
#     #   print("Gompertz: ",row)
#     #   break
     
#     # for row in fuzzy_rank_array_10:
#     #   print("Weibull: ",row)
#     #   break
#     ######
      

#     # print("Gaussian: ", fuzzy_rank_array_6)
#     # print("Mitcherlich: ", fuzzy_rank_array_7)
#     # print("Blumberg: ",fuzzy_rank_array_8)
#     # print("Gompertz: ", fuzzy_rank_array_9)
#     # print("Richard: ", fuzzy_rank_array_10)
#     # print("\n")

#     merged_fuzzy_rank_array_2 = MergeFuzzyRanks(fuzzy_rank_array_6, fuzzy_rank_array_7, fuzzy_rank_array_8, 
#                                               fuzzy_rank_array_9, fuzzy_rank_array_10, confidence_factor_array_2)
#     # print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_2)


    
#     # For classifier 3
#     confidence_factor_array_3, Y_Test = Train(X_Train, Y_Train, X_Test, Y_Test, clf6)
#     fuzzy_rank_array_11 = GenerateFuzzyRank_Gaussian(confidence_factor_array_3)
#     fuzzy_rank_array_12 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_3)
#     fuzzy_rank_array_13 = GenerateFuzzyRank_Blumberg(confidence_factor_array_3)
#     fuzzy_rank_array_14 = GenerateFuzzyRank_Gompertz(confidence_factor_array_3)
#     fuzzy_rank_array_15 = GenerateFuzzyRank_Weibull(confidence_factor_array_3)
    
#     ######
#     # print("\nCLASSIFIER 3")
#     # for row in fuzzy_rank_array_11:
#     #   print("Gaussian: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_12:
#     #   print("Mitcherlich: ",row)
#     #   break
     
#     # for row in fuzzy_rank_array_13:
#     #   print("Blumberg: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_14:
#     #   print("Gompertz: ",row)
#     #   break
      
#     # for row in fuzzy_rank_array_15:
#     #   print("Weibull: ",row)
#     #   break
#     ######
      
#     # print("Gaussian: ", fuzzy_rank_array_11)
#     # print("Mitcherlich: ", fuzzy_rank_array_12)
#     # print("Blumberg: ",fuzzy_rank_array_13)
#     # print("Gompertz: ", fuzzy_rank_array_14)
#     # print("Richard: ", fuzzy_rank_array_15)
#     # print("\n")


#     merged_fuzzy_rank_array_3 = MergeFuzzyRanks(fuzzy_rank_array_11, fuzzy_rank_array_12, fuzzy_rank_array_13, 
#                                               fuzzy_rank_array_14, fuzzy_rank_array_15, confidence_factor_array_3)
#     # print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_3)


    
#     # # For classifier 4
#     # confidence_factor_array_4, Y_Test = Train(X_Train, Y_Train, X_Test, Y_Test, clf5)
#     # fuzzy_rank_array_4 = GenerateFuzzyRank(confidence_factor_array_4)

#     # # For classifier 5
#     # confidence_factor_array_5, Y_Test = Train(X_Train, Y_Train, X_Test, Y_Test, clf5)
#     # fuzzy_rank_array_5 = GenerateFuzzyRank(confidence_factor_array_5)


#     # Ensembling all
#     ## For 5 classifiers
#     # Y_Pred = PredictClass_5(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3, fuzzy_rank_array_4, fuzzy_rank_array_5, 
#     #                      confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3, confidence_factor_array_4, confidence_factor_array_5)
#     ## For 3 classifiers
#     Y_Pred = PredictClass_3(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, merged_fuzzy_rank_array_3,
#                         confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3)


#     # Results
#     # Correct = 0
#     # Wrong = 0

#     # for i in range(len(Y_Test)):
#     #   if Y_Test[i] == Y_Pred[i]:
#     #     Correct += 1
#     #   else:
#     #     Wrong += 1


#     ## Do Comment this break
#     # print("Actual Class: ", Y_Test[i])
#     # break

    

#     # Uncomment for Test
#     acc_fold = accuracy_score(Y_Test, Y_Pred)
#     avg_acc_list.append(acc_fold)
#     recall_fold = recall_score(Y_Test, Y_Pred, average='macro')
#     avg_recall_list.append(recall_fold)
#     f1_fold  = f1_score(Y_Test, Y_Pred, average='macro')
#     avg_f1_list.append(f1_fold)

#     # print("*************Correct: ",Correct,"Wrong: ",Wrong,"Accuracy: ", metrics.accuracy_score(Y_Test, Y_Pred)*100)
#     # avg_acc.append((Correct*100)/(Correct+Wrong))
#     print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
#     print('________________________________________________________________')

#   avg_acc = np.asarray(avg_acc_list)
#   avg_recall = np.asarray(avg_recall_list)
#   avg_f1 = np.asarray(avg_f1_list)
#   # print("\nOverall Accuracy: ", sum(avg_acc)/len(avg_acc))
#   print("\n")
#   print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(np.mean(avg_acc), np.mean(avg_recall), np.mean(avg_f1)))


In [ ]:
# # Code for testing single ML classifiers -- Uncomment and Test!!!!

# if __name__ == "__main__":  
#   X,Y,folds = DataPreparation('/content/drive/MyDrive/SNOW/WISDM.npz')
#   clf1 = DTC()
#   clf2 = KNC()
#   clf3 = LRC()
#   clf4 = RForest()
#   clf5 = AdaBoost()
#   clf6 = RBFC()

#   avg_acc_list = []
#   avg_recall_list = []
#   avg_f1_list = []

#   clf = RBFC()

#   avg_acc_list = []
#   avg_recall_list = []
#   avg_f1_list = []

#   for i in range(len(folds)):
#     Train_idx = folds[i][0]
#     Test_idx = folds[i][1]
#     X_Train_nonnormalized = X[Train_idx]
#     Y_Train = Y[Train_idx]
#     X_Test_nonnormalized = X[Test_idx]
#     Y_Test = Y[Test_idx]

#     X_Train_nonnormalized = feature_extraction(X_Train_nonnormalized)
#     X_Train = preprocessing.normalize(X_Train_nonnormalized, norm='l1')
#     X_Test_nonnormalized = feature_extraction(X_Test_nonnormalized)
#     X_Test = preprocessing.normalize(X_Test_nonnormalized, norm='l1')


#     clf.fit(X_Train, Y_Train)
#     f1 = clf.predict(X_Test)  

#     acc_fold = accuracy_score(Y_Test, f1)
#     avg_acc_list.append(acc_fold)
#     recall_fold = recall_score(Y_Test, f1, average='macro')
#     avg_recall_list.append(recall_fold)
#     f1_fold  = f1_score(Y_Test, f1, average='macro')
#     avg_f1_list.append(f1_fold)

#     # print("*************Correct: ",Correct,"Wrong: ",Wrong,"Accuracy: ", metrics.accuracy_score(Y_Test, Y_Pred)*100)
#     # avg_acc.append((Correct*100)/(Correct+Wrong))
#     print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
#     print('________________________________________________________________')

#   avg_acc = np.asarray(avg_acc_list)
#   avg_recall = np.asarray(avg_recall_list)
#   avg_f1 = np.asarray(avg_f1_list)
#   # print("\nOverall Accuracy: ", sum(avg_acc)/len(avg_acc))
#   print("\n")
#   print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(np.mean(avg_acc), np.mean(avg_recall), np.mean(avg_f1)))


In [ ]:
# # Code for testing ensembled DNN classifiers -- Uncomment and Test!!!!

# if __name__ == "__main__":  
#   X,y,folds,n_class = DataPreparation_DNN('/content/drive/MyDrive/data/SNOW/WHARF.npz')

#   avg_acc_list = []
#   avg_recall_list = []
#   avg_f1_list = []

#   for i in range(0, len(folds)):
#     train_idx = folds[i][0]
#     test_idx = folds[i][1]
#     X_train = X[train_idx]
#     X_test = X[test_idx]
#     y_train = y[train_idx]
#     y_test = y[test_idx]

#     # For classifier 1
#     confidence_factor_array_1 = Train_JGH(X, y, X_train, X_test, y_train, y_test, n_class)
#     fuzzy_rank_array_1 = GenerateFuzzyRank_Gaussian(confidence_factor_array_1)
#     fuzzy_rank_array_2 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_1)
#     fuzzy_rank_array_3 = GenerateFuzzyRank_Blumberg(confidence_factor_array_1)
#     fuzzy_rank_array_4 = GenerateFuzzyRank_Gompertz(confidence_factor_array_1)
#     fuzzy_rank_array_5 = GenerateFuzzyRank_Weibull(confidence_factor_array_1)

#     merged_fuzzy_rank_array_1 = MergeFuzzyRanks(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3, 
#                                               fuzzy_rank_array_4, fuzzy_rank_array_5, confidence_factor_array_1)
    


#     # For classifier 2
#     confidence_factor_array_2 = Train_Lyu(X, y, X_train, X_test, y_train, y_test, n_class)
#     fuzzy_rank_array_6 = GenerateFuzzyRank_Gaussian(confidence_factor_array_2)
#     fuzzy_rank_array_7 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_2)
#     fuzzy_rank_array_8 = GenerateFuzzyRank_Blumberg(confidence_factor_array_2)
#     fuzzy_rank_array_9 = GenerateFuzzyRank_Gompertz(confidence_factor_array_2)
#     fuzzy_rank_array_10 = GenerateFuzzyRank_Weibull(confidence_factor_array_2)

#     merged_fuzzy_rank_array_2 = MergeFuzzyRanks(fuzzy_rank_array_6, fuzzy_rank_array_7, fuzzy_rank_array_8, 
#                                               fuzzy_rank_array_9, fuzzy_rank_array_10, confidence_factor_array_2)


    
#     # For classifier 3
#     confidence_factor_array_3 = Train_ResNet(X, y, X_train, X_test, y_train, y_test, n_class)
#     fuzzy_rank_array_11 = GenerateFuzzyRank_Gaussian(confidence_factor_array_3)
#     fuzzy_rank_array_12 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_3)
#     fuzzy_rank_array_13 = GenerateFuzzyRank_Blumberg(confidence_factor_array_3)
#     fuzzy_rank_array_14 = GenerateFuzzyRank_Gompertz(confidence_factor_array_3)
#     fuzzy_rank_array_15 = GenerateFuzzyRank_Weibull(confidence_factor_array_3)

#     merged_fuzzy_rank_array_3 = MergeFuzzyRanks(fuzzy_rank_array_11, fuzzy_rank_array_12, fuzzy_rank_array_13, 
#                                               fuzzy_rank_array_14, fuzzy_rank_array_15, confidence_factor_array_3)



#     y_pred = PredictClass_3(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, merged_fuzzy_rank_array_3,
#                             confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3)

#     # Correct format of y_test (becomes 1D)
#     y_test = np.argmax(y_test, axis=1)

#     # Uncomment for Test
#     acc_fold = accuracy_score(y_test, y_pred)
#     avg_acc_list.append(acc_fold)
#     recall_fold = recall_score(y_test, y_pred, average='macro')
#     avg_recall_list.append(recall_fold)
#     f1_fold  = f1_score(y_test, y_pred, average='macro')
#     avg_f1_list.append(f1_fold)

#     # print("*************Correct: ",Correct,"Wrong: ",Wrong,"Accuracy: ", metrics.accuracy_score(Y_Test, Y_Pred)*100)
#     # avg_acc.append((Correct*100)/(Correct+Wrong))
#     print('Accuracy[{:.4f}] Recall[{:.4f}] F1[{:.4f}] at Fold[{}]'.format(acc_fold, recall_fold, f1_fold ,i+1))
#     print('________________________________________________________________')

#   avg_acc = np.asarray(avg_acc_list)
#   avg_recall = np.asarray(avg_recall_list)
#   avg_f1 = np.asarray(avg_f1_list)
#   # print("\nOverall Accuracy: ", sum(avg_acc)/len(avg_acc))
#   print("\n")
#   print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(np.mean(avg_acc), np.mean(avg_recall), np.mean(avg_f1)))



In [ ]:
# Code for testing Opportunity Dataset -- Uncomment and Test!!

def OpportunityDataSetPreparation():
  X_train=np.load('/content/drive/MyDrive/data/Opportunity/Opportunity_train_X.npz')['arr_0']
  X_test=np.load('/content/drive/MyDrive/data/Opportunity/Opportunity_test_X.npz')['arr_0']
  Y_train=np.load('/content/drive/MyDrive/data/Opportunity/Opportunity_train_y.npz')['arr_0']
  Y_test=np.load('/content/drive/MyDrive/data/Opportunity/Opportunity_test_Y.npz')['arr_0']
  X = np.concatenate((X_train, X_test), axis = 0)
  Y = np.concatenate((Y_train, Y_test), axis = 0)
  classes_number = Y_train.shape[1]
  return X, Y, X_train,Y_train,X_test,Y_test,classes_number


X, Y, X_Train, Y_Train, X_Test, Y_Test, n_classes = OpportunityDataSetPreparation()
#clf1 = DTC()
#clf2 = KNC()
#clf3 = LRC()
#clf4 = RForest()
#clf5 = AdaBoost()
#clf6 = RBFC()
#X_Train = feature_extraction(X_Train)
#X_Train = preprocessing.normalize(X_Train_nonnormalized, norm='l1')
#X_Test = feature_extraction(X_Test)
#X_Test = preprocessing.normalize(X_Test_nonnormalized, norm='l1')


# For classifier 1
confidence_factor_array_1 = Train_Lyu_Opp(X, Y, X_Train, X_Test, Y_Train, Y_Test, n_classes)
fuzzy_rank_array_1 = GenerateFuzzyRank_Gaussian(confidence_factor_array_1)
fuzzy_rank_array_2 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_1)
fuzzy_rank_array_3 = GenerateFuzzyRank_Blumberg(confidence_factor_array_1)
fuzzy_rank_array_4 = GenerateFuzzyRank_Gompertz(confidence_factor_array_1)
fuzzy_rank_array_5 = GenerateFuzzyRank_Weibull(confidence_factor_array_1)

# print(fuzzy_rank_array_1[0])
# print(fuzzy_rank_array_2[0])
# print(fuzzy_rank_array_3[0])
# print(fuzzy_rank_array_4[0])
# print(fuzzy_rank_array_5[0])
# print(confidence_factor_array_1)
# break

merged_fuzzy_rank_array_1 = MergeFuzzyRanks(fuzzy_rank_array_1, fuzzy_rank_array_2, fuzzy_rank_array_3, 
                                              fuzzy_rank_array_4, fuzzy_rank_array_5, confidence_factor_array_1)
# print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_1)

  
# For classifier 2
confidence_factor_array_2 = Train_JGH_Opp(X, Y, X_Train, X_Test, Y_Train, Y_Test, n_classes)
fuzzy_rank_array_6 = GenerateFuzzyRank_Gaussian(confidence_factor_array_2)
fuzzy_rank_array_7 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_2)
fuzzy_rank_array_8 = GenerateFuzzyRank_Blumberg(confidence_factor_array_2)
fuzzy_rank_array_9 = GenerateFuzzyRank_Gompertz(confidence_factor_array_2)
fuzzy_rank_array_10 = GenerateFuzzyRank_Weibull(confidence_factor_array_2)
    
merged_fuzzy_rank_array_2 = MergeFuzzyRanks(fuzzy_rank_array_6, fuzzy_rank_array_7, fuzzy_rank_array_8, 
                                              fuzzy_rank_array_9, fuzzy_rank_array_10, confidence_factor_array_2)
# print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_2)


    
# For classifier 3
confidence_factor_array_3 = Train_ResNet_Opp(X, Y, X_Train, X_Test, Y_Train, Y_Test, n_classes)
fuzzy_rank_array_11 = GenerateFuzzyRank_Gaussian(confidence_factor_array_3)
fuzzy_rank_array_12 = GenerateFuzzyRank_Mitcherlich(confidence_factor_array_3)
fuzzy_rank_array_13 = GenerateFuzzyRank_Blumberg(confidence_factor_array_3)
fuzzy_rank_array_14 = GenerateFuzzyRank_Gompertz(confidence_factor_array_3)
fuzzy_rank_array_15 = GenerateFuzzyRank_Weibull(confidence_factor_array_3)
    
merged_fuzzy_rank_array_3 = MergeFuzzyRanks(fuzzy_rank_array_11, fuzzy_rank_array_12, fuzzy_rank_array_13, 
                                              fuzzy_rank_array_14, fuzzy_rank_array_15, confidence_factor_array_3)
# print("Merged Fuzzy Matrix: ", merged_fuzzy_rank_array_3)


    
    
Y_Pred = PredictClass_3(merged_fuzzy_rank_array_1, merged_fuzzy_rank_array_2, merged_fuzzy_rank_array_3,
                        confidence_factor_array_1, confidence_factor_array_2, confidence_factor_array_3)

Y_Test = np.argmax(y_test, axis=1)    

# Uncomment for Test
avg_acc = accuracy_score(Y_Test, Y_Pred)
avg_recall = recall_score(Y_Test, Y_Pred, average='macro')
avg_f1 = f1_score(Y_Test, Y_Pred, average='macro')

print('Overall Accuracy[{:.4f}] Overall Recall[{:.4f}] Overall F1[{:.4f}]'.format(avg_acc, avg_recall, avg_f1))

